In [1]:
import sys
sys.path.append('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch')
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from essentials import complete_preprocessing_pipeline
from features import GenerateFeatures 
import copy

In [2]:
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_dict_imu.pkl', 'rb') as f:
    imu_dict = pickle.load(f)
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_minze_dict.pkl', 'rb') as f:
    ground_truth_dict = pickle.load(f)
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_dict_urineestimate_method1.pkl', 'rb') as f:
    urine_estimate_dict = pickle.load(f)

Remove the data with the almost the entire data being void.
1. subj_9_void4
2. subj_11_void2

In [3]:
del imu_dict['subj_9_void4']
del imu_dict['subj_11_void2']

Extract windowed features from the 3 axis of the accelerometer, and the 3 axis of the gyroscope.
1. mean
2. rms
3. std
4. range
5. min
6. max
7. time energy
8. permutation entropy
9. spectral entropy
10. spectral_centroid
11. spectral_spread
12. spectral_flatness
13. peak_frequency_ratio
14. hjorth_frequency
15. spectral_skewness
16. spectral_kurtosis
17. snr

In [4]:
data_dict = copy.deepcopy(imu_dict)
labelled_imu_dict = complete_preprocessing_pipeline(data_dict, ground_truth_dict, 
                                target_fs=60,normalize_data=True, use_three_classes=True)

Step 1: Resampling data to 60 Hz...


  0%|          | 0/41 [00:00<?, ?it/s]

Step 2: Processing each instance: 100%|██████████| 41/41 [00:00<00:00, 292.69it/s]


In [5]:
window_sizes = [3,5]
# overlaps = [0.0, 0.5, 0.8]

## Three class data
classes: pre-void, void, non-void

In [6]:
for window_size in window_sizes:
    all_features = []
    for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):
        df = labelled_imu_dict[imu_data]

        analyzer = GenerateFeatures(fs=60, window_duration=window_size, overlap=0.0)
        features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])
            
        table = analyzer.create_summary_table()
        table['experiment_id'] = exp_id + 1
        all_features.append(table)

    final_features = pd.concat(all_features, ignore_index=True)
    print(f'Window size and overlap: {window_size}s')
    final_features.to_csv(f'/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/sheep/extracted_data/{window_size}s.csv', index=False)
    final_features.head()

Extracting sheep paper features: 100%|██████████| 13/13 [00:00<00:00, 906.64it/s]


Window size and overlap: 3s


Extracting sheep paper features: 100%|██████████| 8/8 [00:00<00:00, 1079.58it/s]


Window size and overlap: 5s
